In [ ]:
from decord import VideoReader, cpu
from transformers import VideoMAEImageProcessor, AutoModel, AutoConfig
import numpy as np
import torch

In [ ]:
config = AutoConfig.from_pretrained("OpenGVLab/VideoMAEv2-giant", trust_remote_code=True)
processor = VideoMAEImageProcessor.from_pretrained("OpenGVLab/VideoMAEv2-giant")
model = AutoModel.from_pretrained("OpenGVLab/VideoMAEv2-giant", config=config, trust_remote_code=True)

In [ ]:
video_path = "/data/ephemeral/home/MovieclipsAkeelah and the Bee (59) Movie CLIP - Big Words Come From Little Words (2006) HD-_UZxXUwQX84.mp4"  # 비디오 파일 경로
vr = VideoReader(video_path, ctx=cpu(0))

In [ ]:
num_frames = 32
frame_indices = np.linspace(0, len(vr) - 1, num_frames, dtype=int)
frames = vr.get_batch(frame_indices).asnumpy()

In [ ]:
frames = [frame.transpose(2, 0, 1) for frame in frames]  # (H, W, C) -> (C, H, W)
inputs = processor(list(frames), return_tensors="pt")
inputs['pixel_values'] = inputs['pixel_values'].permute(0, 2, 1, 3, 4)  # B, T, C, H, W -> B, C, T, H, W

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

# 출력 확인
print(outputs)